# 로지스틱 회귀 적용

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lg-aimers/sample_submission.csv
/kaggle/input/lg-aimers/train.csv
/kaggle/input/lg-aimers/test.csv


In [2]:
data_path = '/kaggle/input/lg-aimers/'

train = pd.read_csv(data_path + 'train.csv', index_col= 'ID')
test = pd.read_csv(data_path + 'test.csv', index_col= 'ID')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col= 'ID')

In [3]:
all_data = pd.concat([train, test], ignore_index= True)
all_data = all_data.drop('임신 성공 여부', axis= 1)

In [4]:
drop_features = []

In [5]:
# "특정 시술 유형" 제거
# "주요 시술 유형" 생성

cat_features = [ 
    "시술 당시 나이", # FI top4
    "시술 유형",
    "주요 시술 유형",
    "배란 유도 유형",
    "난자 출처", # FI top
        
    "난자 기증자 나이", #FI top (알수 없음)

    "정자 출처",
    "정자 기증자 나이",
    "시술 시기 코드", 
    "배아 생성 주요 이유"
]

print(len(cat_features))

10


In [6]:
# "IVF 시술 횟수", "IVF 임신 횟수", "IVF 출산 횟수" 제거
# "DI 시술 횟수", "DI 임신 횟수", "DI 출산 횟수" 제거
# "IVF 임신 성공률", "IVF 출산 성공률", "IVF 실패 횟수", "IVF 유산 횟수", "DI 임신 성공률", "DI 출산 성공률", "DI 실패 횟수", "DI 유산 횟수" 생성 

ord_features = [
    "IVF 임신 성공률", 
    "IVF 출산 성공률", 
    "IVF 실패 횟수", 
    "IVF 유산 횟수", 
    "DI 임신 성공률", 
    "DI 출산 성공률", 
    "DI 실패 횟수", 
    "DI 유산 횟수",
    
    "클리닉 내 총 시술 횟수", #FI top
    "총 시술 횟수",
    "총 임신 횟수",
    "총 출산 횟수"
]

print(len(ord_features))

# drop_features = drop_features + ["총 시술 횟수", "총 임신 횟수", "총 출산 횟수"]

12


In [7]:
num_features = [
    "이식된 배아 수", # FI top1
    "해동 난자 수", #FI top
    "수집된 신선 난자 수", #FI top7
    "파트너 정자와 혼합된 난자 수", #FI top
    "배아 이식 경과일", # FI top2,
    
    
    "저장된 배아 수", # FI top3
    "미세주입 후 저장된 배아 수", #FI top
    "총 생성 배아 수", # FI top5
    "미세주입된 난자 수", #FI top8
    "혼합된 난자 수", #FI top
    "미세주입에서 생성된 배아 수", #FI top
    "미세주입 배아 이식 수", #FI top
    "해동된 배아 수", #FI top
    "난자 채취 경과일", #FI top
    "임신 시도 또는 마지막 임신 경과 연수", #FI top

    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 해동 경과일",
    "저장된 신선 난자 수",
    "기증자 정자와 혼합된 난자 수"
]

print(len(num_features))

# drop_features = drop_features + ["난자 해동 경과일", "난자 혼합 경과일", "배아 해동 경과일", "저장된 신선 난자 수", "기증자 정자와 혼합된 난자 수"]

20


In [8]:
# "남성 주 불임 원인", "남성 부 불임 원인", "불임 원인 - 남성 요인", "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성", "불임 원인 - 정자 형태",
# "여성 주 불임 원인", "여성 부 불임 원인", "불임 원인 - 난관 질환", "불임 원인 - 배란 장애", "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증" 제거
# "부부 주 불임 원인", "부부 부 불임 원인" 제거
# "남성 불임 원인", "여성 불임 원인", "부부 불임 원인" 생성
# "PGD 시술 여부", "PGS 시술 여부" 제거 

bin_features = [
    '배란 자극 여부',
    '단일 배아 이식 여부', #FI top
    '착상 전 유전 진단 사용 여부',
    '남성 불임 원인',
    '여성 불임 원인',
    '부부 불임 원인',
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",

    "불명확 불임 원인",
    "기증 배아 사용 여부",
    '착상 전 유전 검사 사용 여부',
    "대리모 여부"
]

print(len(bin_features))

drop_features = drop_features + ["PGD 시술 여부", "PGS 시술 여부"]

12


In [9]:
# 시술 유형(cat feature) 피처 엔지니어링 

all_data["주요 시술 유형"] = all_data["특정 시술 유형"].apply(lambda x: 
    "ICSI" if "ICSI" in str(x) else 
    "IVF" if "IVF" in str(x) else 
    "IUI" if "IUI" in str(x) else 
    "Other"
)

drop_features = drop_features + ['특정 시술 유형']

all_data['주요 시술 유형'].value_counts(normalize=True, dropna=False) * 100

주요 시술 유형
ICSI     50.208130
IVF      36.909168
Other    10.511578
IUI       2.371124
Name: proportion, dtype: float64

In [10]:
# 과거 이력(ord feature) 피처 엔지니어링

past_features = ["IVF 시술 횟수", "DI 시술 횟수", "IVF 임신 횟수", "DI 임신 횟수", "IVF 출산 횟수", "DI 출산 횟수"]

for col in ["총 시술 횟수", "총 임신 횟수", "총 출산 횟수", "클리닉 내 총 시술 횟수"]:
    all_data[col] = all_data[col].str.replace("회 이상", "").str.replace("회", "").astype(float)
        
for col in past_features:
    all_data[col] = all_data[col].str.replace("회 이상", "").str.replace("회", "").astype(float)

all_data["IVF 임신 성공률"] = all_data["IVF 임신 횟수"] / (all_data["IVF 시술 횟수"] + 1)
all_data["IVF 출산 성공률"] = all_data["IVF 출산 횟수"] / (all_data["IVF 임신 횟수"] + 1)
all_data["IVF 실패 횟수"] = all_data["IVF 시술 횟수"] - all_data["IVF 임신 횟수"]
all_data["IVF 유산 횟수"] = all_data["IVF 임신 횟수"] - all_data["IVF 출산 횟수"]

all_data["DI 임신 성공률"] = all_data["DI 임신 횟수"] / (all_data["DI 시술 횟수"] + 1)
all_data["DI 출산 성공률"] = all_data["DI 출산 횟수"] / (all_data["DI 임신 횟수"] + 1)
all_data["DI 실패 횟수"] = all_data["DI 시술 횟수"] - all_data["DI 임신 횟수"]
all_data["DI 유산 횟수"] = all_data["DI 임신 횟수"] - all_data["DI 출산 횟수"]

drop_features = drop_features + past_features

In [11]:
# 불임 원인(bin features) 피처 엔지니어링 

infertility_features = ["남성 주 불임 원인", "남성 부 불임 원인", "여성 주 불임 원인", "여성 부 불임 원인",
                        "불임 원인 - 남성 요인", "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성", "불임 원인 - 정자 형태",
                        "불임 원인 - 난관 질환", "불임 원인 - 배란 장애", "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증",
                        "부부 주 불임 원인", "부부 부 불임 원인"]

all_data["남성 불임 원인"] = (
    all_data["남성 주 불임 원인"] + 
    all_data["남성 부 불임 원인"] + 
    all_data["불임 원인 - 남성 요인"] + 
    all_data["불임 원인 - 정자 농도"] + 
    all_data["불임 원인 - 정자 면역학적 요인"] + 
    all_data["불임 원인 - 정자 운동성"] + 
    all_data["불임 원인 - 정자 형태"]
).apply(lambda x: 1 if x > 0 else 0)

all_data["여성 불임 원인"] = (
    all_data["여성 주 불임 원인"] + 
    all_data["여성 부 불임 원인"] + 
    all_data["불임 원인 - 난관 질환"] + 
    all_data["불임 원인 - 배란 장애"] + 
    all_data["불임 원인 - 여성 요인"] + 
    all_data["불임 원인 - 자궁경부 문제"] + 
    all_data["불임 원인 - 자궁내막증"]
).apply(lambda x: 1 if x > 0 else 0)

all_data["부부 불임 원인"] = (
    all_data["부부 주 불임 원인"] + 
    all_data["부부 부 불임 원인"]
).apply(lambda x: 1 if x > 0 else 0)

drop_features = drop_features + infertility_features

print(all_data['남성 불임 원인'].value_counts(normalize=True, dropna=False) * 100)
print(all_data['여성 불임 원인'].value_counts(normalize=True, dropna=False) * 100)
print(all_data['부부 불임 원인'].value_counts(normalize=True, dropna=False) * 100)

남성 불임 원인
0    60.57855
1    39.42145
Name: proportion, dtype: float64
여성 불임 원인
0    66.416583
1    33.583417
Name: proportion, dtype: float64
부부 불임 원인
0    95.81546
1     4.18454
Name: proportion, dtype: float64


In [12]:
extracted_data = all_data.drop(columns=drop_features, errors='ignore')

In [13]:
extracted_features = extracted_data.columns
extracted_features

Index(['시술 시기 코드', '시술 당시 나이', '임신 시도 또는 마지막 임신 경과 연수', '시술 유형', '배란 자극 여부',
       '배란 유도 유형', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부',
       '불명확 불임 원인', '배아 생성 주요 이유', '총 시술 횟수', '클리닉 내 총 시술 횟수', '총 임신 횟수',
       '총 출산 횟수', '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수',
       '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수',
       '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수',
       '기증자 정자와 혼합된 난자 수', '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이',
       '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부', '난자 채취 경과일',
       '난자 해동 경과일', '난자 혼합 경과일', '배아 이식 경과일', '배아 해동 경과일', '주요 시술 유형',
       'IVF 임신 성공률', 'IVF 출산 성공률', 'IVF 실패 횟수', 'IVF 유산 횟수', 'DI 임신 성공률',
       'DI 출산 성공률', 'DI 실패 횟수', 'DI 유산 횟수', '남성 불임 원인', '여성 불임 원인',
       '부부 불임 원인'],
      dtype='object')

In [14]:
len(extracted_features)

54

In [15]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder()

encoded_cat_matrix = onehot_encoder.fit_transform(extracted_data[cat_features])

encoded_cat_matrix

<346418x57 sparse matrix of type '<class 'numpy.float64'>'
	with 3464180 stored elements in Compressed Sparse Row format>

In [16]:
remaining_features = list(set(extracted_features) - set(cat_features))

In [17]:
print(extracted_data[remaining_features].dtypes)

난자 채취 경과일                float64
DI 실패 횟수                 float64
미세주입에서 생성된 배아 수          float64
동결 배아 사용 여부              float64
배아 이식 경과일                float64
총 시술 횟수                  float64
미세주입된 난자 수               float64
IVF 출산 성공률               float64
신선 배아 사용 여부              float64
난자 해동 경과일                float64
착상 전 유전 진단 사용 여부         float64
총 출산 횟수                  float64
배란 자극 여부                   int64
저장된 신선 난자 수              float64
대리모 여부                   float64
착상 전 유전 검사 사용 여부         float64
해동된 배아 수                 float64
단일 배아 이식 여부              float64
IVF 유산 횟수                float64
임신 시도 또는 마지막 임신 경과 연수    float64
부부 불임 원인                   int64
미세주입 배아 이식 수             float64
수집된 신선 난자 수              float64
배아 해동 경과일                float64
미세주입 후 저장된 배아 수          float64
이식된 배아 수                 float64
IVF 임신 성공률               float64
IVF 실패 횟수                float64
불명확 불임 원인                  int64
해동 난자 수                  float64
총 생성 배아 수 

In [18]:
from sklearn.preprocessing import MinMaxScaler

# min-max 정규화
extracted_data[ord_features] = MinMaxScaler().fit_transform(extracted_data[ord_features].fillna(0))
extracted_data[num_features] = MinMaxScaler().fit_transform(extracted_data[num_features].fillna(0))

In [19]:
from scipy import sparse

# 인코딩 및 스케일링된 피처 합치기
all_data_sprs = sparse.hstack([sparse.csr_matrix(extracted_data[remaining_features].fillna(0).values),
                               encoded_cat_matrix],
                              format='csr')

In [20]:
num_train = len(train) # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
X_train = all_data_sprs[:num_train] # 0 ~ num_train - 1행
X_test = all_data_sprs[num_train:] # num_train ~ 마지막 행

y = train['임신 성공 여부']

In [21]:
X_train.shape[0]

256351

In [22]:
y.shape[0]

256351

In [23]:
from sklearn.model_selection import train_test_split

# 훈련 데이터, 검증 데이터 분리
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y,
                                                      test_size=0.1,
                                                      stratify=y,
                                                      random_state=10)

In [24]:
%%time

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# 로지스틱 회귀 모델 생성
logistic_model = LogisticRegression()

# 하이퍼파라미터 값 목록
lr_params = {'C':[0.1, 0.125, 0.2], 'max_iter':[800, 900, 1000], 
             'solver':['liblinear'], 'random_state':[42]}

# 그리드서치 객체 생성
gridsearch_logistic_model = GridSearchCV(estimator=logistic_model,
                                         param_grid=lr_params,
                                         scoring='roc_auc', # 평가지표
                                         cv=5)
# 그리드서치 수행
gridsearch_logistic_model.fit(X_train, y_train)

print('최적 하이퍼파라미터:', gridsearch_logistic_model.best_params_)

최적 하이퍼파라미터: {'C': 0.2, 'max_iter': 800, 'random_state': 42, 'solver': 'liblinear'}
CPU times: user 3min 10s, sys: 3.17 s, total: 3min 13s
Wall time: 3min 13s


In [ ]:
# from sklearn.linear_model import LogisticRegression

# logistic_model = LogisticRegression(max_iter=1000, random_state=42) # 모델 생성
# logistic_model.fit(X_train, y_train) # 모델 훈련

In [25]:
y_valid_preds = gridsearch_logistic_model.predict_proba(X_valid)[:, 1]

In [26]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수

# 검증 데이터 ROC AUC
roc_auc = roc_auc_score(y_valid, y_valid_preds)

print(f'검증 데이터 ROC AUC : {roc_auc}')

검증 데이터 ROC AUC : 0.7240389906541294
